## 1. NumPy Vectorization

- numpy 벡터화는 기본적으로 `np.ndarray`를 대상으로 함

In [6]:
import numpy as np

x_list = [1, 2, 3]
print(x_list*2) # 파이썬 리스트 - 리스트 복제

x = np.array(x_list)
print(x*2) # numpy 배열 - 원소별 연산

[1, 2, 3, 1, 2, 3]
[2 4 6]


1) ufunc: 원소별 연산을 for문 없이 구현
- numpy의 ufunc(universal function)는 배열 원소별 연산을 C level에서 처리하게 해주는 기본 도구이므로 for문보다 빠르다!

In [7]:
# 1) 수학/비교/논리 연산은 대부분 ufunc

x = np.array([1.0, 4.0, 9.0])

print(np.sqrt(x))            # [1. 2. 3.]
print(np.log(x))             # [0.         1.38629436 2.19722458]
print(x > 3)                 # [False  True  True]
print(np.where(x > 3, 1, 0)) # [0 1 1]

[1. 2. 3.]
[0.         1.38629436 2.19722458]
[False  True  True]
[0 1 1]


In [8]:
# ReLU 함수 구현
x = np.array([-3, -1, 0, 2, 5])
y = np.maximum(x, 0) # if 없이 벡터화
print(y)

[0 0 0 2 5]


2) Broadcasting: 모양(shape)이 달라도 자동으로 맞춰 연산
- 핵심 규칙: 뒤 차원부터 비교해서
    - 두 차원이 같거나
    - 둘 중 하나가 1이면
    호환됨!

In [11]:
# (N, D) ± (D,)
X = np.random.randn(5, 3)   # (5, 3)
print(X)
mu = X.mean(axis=0)         # (3,)
print(mu)
X_centered = X - mu         # mu가 (5, 3)으로 broadcast
print(X_centered)
print(X.shape, mu.shape, X_centered.shape)

[[-0.91009788 -0.07388561  0.37987671]
 [-0.4815046   0.29419912  0.2965512 ]
 [-1.15978472 -1.01129191 -2.12807028]
 [-0.06942    -0.19358232 -1.09222469]
 [-1.61804038  1.7773785  -0.79827284]]
[-0.84776951  0.15856356 -0.66842798]
[[-0.06232836 -0.23244917  1.04830469]
 [ 0.36626491  0.13563556  0.96497918]
 [-0.3120152  -1.16985547 -1.4596423 ]
 [ 0.77834952 -0.35214587 -0.42379671]
 [-0.77027087  1.61881494 -0.12984486]]
(5, 3) (3,) (5, 3)


In [12]:
# (N,)을 (N,1)로 만들어서 (N, M)과 연산하기
a = np.array([1, 2, 3]) # (3,)
b = np.array([10, 20, 30, 40]) # (4,)

# a[: None] => (3, 1), b[None, :] => (1, 4)
grid = a[:, None] + b[None, :]
print(grid.shape)
print(grid)

(3, 4)
[[11 21 31 41]
 [12 22 32 42]
 [13 23 33 43]]


3) Indexing: 슬라이싱 + boolean mask + fancy index

In [14]:
# 1. 기본 슬라이싱
X = np.arange(20).reshape(4, 5) # (4, 5)
print(np.arange(20))
print(X)
print(X[:, 2:4])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]
[[ 2  3]
 [ 7  8]
 [12 13]
 [17 18]]


In [15]:
# 2. boolean mask로 필터링/치환
X = np.array([[-1, 2, 3],
              [4, -5, 6]])
mask = (X < 0)
print(mask)

X2 = X.copy()
X2[mask] = 0
print(X2)

[[ True False False]
 [False  True False]]
[[0 2 3]
 [4 0 6]]


In [19]:
# 3. 조건 + 조건 (and, or) 조합
x = np.array(np.arange(1, 10))
mask = (x >= 3) & (x <= 7) # 파이썬의 and가 아니라 &를 사용!
print(x[mask])

[3 4 5 6 7]


In [20]:
# 4. Fancy indexing: 원하는 인덱스만 뽑기
x = np.array([10, 20, 30, 40, 50])
idx = np.array([0, 3, 3, 1])
print(x[idx])

[10 40 40 20]


4) Axis(축) 기반 집계: sum/mean/max로 for문 제거
- for문 없이 n차원 배열에서 함수 적용/axis 이해하기
    - axis=0: "행 방향으로 줄여서(=열별 집계)"
    - axis=1: "열 방향으로 줄여서(=행별 집계)"

In [21]:
X = np.arange(12).reshape(3, 4)

print(X.sum(axis=0)) # 열 합: (4,)
print(X.sum(axis=1)) # 행 합: (3,)

print(X.mean(axis=1)) # 행 평균: (3,)
print(X.max(axis=0))  # 열 최대값: (4,)

[12 15 18 21]
[ 6 22 38]
[1.5 5.5 9.5]
[ 8  9 10 11]


5) reshape/transpose: 모양을 바꿔서 벡터화가 되게 만들기
- reshape는 데이터는 바꾸지 않고 모양만 바꿈 (원소 개수는 유지)
- broadcasting이 안 될 때 (N,)을 (N,1)로 reshape 하는 게 전형적인 해결책

In [22]:
x = np.arange(12)
A = x.reshape(3, 4)
B = A.T

print(A.shape, B.shape)

(3, 4) (4, 3)


6. ETC numpy vectorize skills

In [ ]:
# 1. 조건 분기 벡터화 - np.where / np.select
x = np.array([-2, -1, 0, 1, 2])
y = np.where(x > 0, x, 0) # if x > 0 return x else return 0
print(y)

[0 0 0 1 2]


In [ ]:
# 2. 행마다 다른 인덱스로 뽑기 - np.take_along_axis
# ex) batch별 argmax 위치 값 뽑기
X = np.array([[1, 9, 3],
              [7, 2, 8]])
idx = np.array([[1],
                [2]])  # (N, 1)
out = np.take_along_axis(X, idx, axis=1) # (N, 1)
print(out.ravel()) # [9 8]

[9 8]


In [ ]:
# 3. 구간(bin) 매핑을 for문 없이 - np.searchsorted / np.digitize
edges = np.array([0, 10, 20, 30])
x = np.array([3, 17, 27, 29])
bin_id = np.digitize(x, edges) - 1
print(bin_id) # [0 1 2 2]

[0 1 2 2]


In [ ]:
# 4. 다차원 곱/축 합을 한 줄로 - np.einsum / matmul / tensordot
# pairwise dot, 가중합, 배치 행렬곱 등에서 for문 제거
A = np.random.randn(5, 3)
B = np.random.randn(5, 3)
print(A)
print(B)
dot = np.einsum('nd,nd->n', A, B) # 각 행의 dot: (5,)

# 각 행(row)을 독립적인 벡터로 보고 대응하는 행끼리 dot product 수행
# 두 입력 배열의 행 수와 동일한 길이의 1차원 배열 반환
print(dot)

[[ 0.21946471  0.91494708 -0.02927509]
 [-1.07513621  0.34412369 -0.32883659]
 [ 1.07092064  0.71124095 -0.17486011]
 [-0.93450915 -0.84330364 -0.73102974]
 [-1.08903744  1.49715056 -0.78400969]]
[[ 0.01897095  0.8727142  -0.43942211]
 [ 0.46391088 -2.26759528  1.71744067]
 [ 0.17665819 -0.12149574  0.3904708 ]
 [-0.96478881  1.71411718 -1.49935351]
 [ 0.31520855  0.11252911 -1.40646561]]
[ 0.81551489 -1.84385799  0.03449639  0.55215473  0.92788177]


In [41]:
vec = np.random.rand(100, 100)
print("np.sum:", np.sum(vec))
%timeit np.sum(vec)
print("np.einsum:", np.einsum('ij->', vec))
%timeit np.einsum('ij->', vec)

np.sum: 5009.025489722975
2.14 μs ± 5.44 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
np.einsum: 5009.0254897229715
1.46 μs ± 6.62 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [43]:
# 5. 슬라이딩 윈도우 - sliding_window_view
from numpy.lib.stride_tricks import sliding_window_view
x = np.arange(10)
w = sliding_window_view(x, window_shape=4)[::2] # stride 2
print(w.shape)
print(w)

(4, 4)
[[0 1 2 3]
 [2 3 4 5]
 [4 5 6 7]
 [6 7 8 9]]


## 2. PyTorch Vectorization

- PyTorch는 Tensor = ndarray + autograd + GPU

1) Broadcasting + 기본 원소 연산
- numpy와 완전 동일하며, unsqueeze/None 인덱싱이 자주 등장함

In [ ]:
import torch

X = torch.randn(32, 128)    # (B, D)
mu = X.mean(dim=0)          # (D,)
X_centered = X - mu         # broadcast

print(X.shape)
print(mu.shape)
print(X_centered.shape)

torch.Size([32, 128])
torch.Size([128])
torch.Size([32, 128])


In [10]:
# 원하는 축에 차원 추가하기 - None / unsqueeze
a = torch.arange(3) # (3,)
b = torch.arange(4) # (4,)
grid = a[:, None] + b[None, :]  # (3, 4)

print(a)
print(b)
print(grid)

A0 = torch.unsqueeze(a, dim=0)
print(A0)
A1 = torch.unsqueeze(a, dim=1)
print(A1)

tensor([0, 1, 2])
tensor([0, 1, 2, 3])
tensor([[0, 1, 2, 3],
        [1, 2, 3, 4],
        [2, 3, 4, 5]])
tensor([[0, 1, 2]])
tensor([[0],
        [1],
        [2]])


2) 마스킹/조건 분기: `where`, `masked_fill`, boolean indexing
- 전처리 및 loss 계산에 매우 많이 사용됨

In [15]:
x = torch.tensor([-2., -1., 0., 1., 2.])
relu = torch.clamp(x, min=0)
print(relu)
relu2 = torch.where(x > 0, x, torch.zeros_like(x))
print(relu2)

tensor([0., 0., 0., 1., 2.])
tensor([0., 0., 0., 1., 2.])


In [17]:
X = torch.randn(4, 3)
mask = X < 0
X2 = X.masked_fill(mask, 0.0) # 음수만 0
print(X)
print(mask)
print(X2)

tensor([[ 0.5927,  2.1155,  0.6215],
        [-0.2331,  1.5869,  0.6540],
        [-1.1697,  1.5475,  1.3878],
        [ 0.5975,  0.2010, -0.4702]])
tensor([[False, False, False],
        [ True, False, False],
        [ True, False, False],
        [False, False,  True]])
tensor([[0.5927, 2.1155, 0.6215],
        [0.0000, 1.5869, 0.6540],
        [0.0000, 1.5475, 1.3878],
        [0.5975, 0.2010, 0.0000]])


3) 축 집계(reduction): sum/mean/max/... + keepdim
- batch loss, statistics, score, etc.

In [21]:
X = torch.randn(8, 5)
row_sum = X.sum(dim=1)      # (8,)
col_mean = X.mean(dim=0)    # (5,)
mx = X.max(dim=1).values    # (8,)

print(row_sum.shape)
print(col_mean.shape)
print(mx.shape)

torch.Size([8])
torch.Size([5])
torch.Size([8])


In [ ]:
# keepdim=True면 브로드캐스팅 맞추기 편함
X = torch.randn(8, 5)
mu1 = X.mean(dim=1, keepdim=False)
mu2 = X.mean(dim=1, keepdim=True)  # (8, 1)
X_norm = X - mu2                   # broadcast

print(mu1.shape) # keepdim=False이면 차원 제거
print(mu2.shape) # keepdim=True이면 차원 유지
print(X_norm.shape)

torch.Size([8])
torch.Size([8, 1])
torch.Size([8, 5])


4) (중요) 배치별로 다른 인덱스 뽑기/쓰기 : `gather` / `scatter`

In [34]:
# gather: 배치별 정답 label 위치의 logit만 뽑기
B, C = 4, 6
logits = torch.randn(B, C)
print(logits)
labels = torch.tensor([1, 3, 0, 5]) # (B,)

picked = logits.gather(1, labels[:, None]).squeeze(1) # (B,)
print(picked)

tensor([[ 0.8128, -1.5767,  0.6766,  1.6483,  1.5687, -0.6723],
        [-0.2185,  1.4773, -0.2952,  0.2396,  1.0336, -2.0708],
        [ 0.2267, -0.6256,  0.9033,  0.9330, -1.8699,  0.8545],
        [-0.3218,  0.7439, -1.8198,  0.1595,  0.9763,  1.9887]])
tensor([-1.5767,  0.2396,  0.2267,  1.9887])


In [33]:
# scatter_: one-hot 만들기
onehot = torch.zeros(B, C)
out = onehot.scatter_(1, labels[:, None], 1.0) # (B, C)
print(out)

tensor([[0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.]])


5) 슬라이딩 윈도우/시퀀스 벡터화: `unfold`

In [38]:
T, F = 100, 9
x = torch.randn(T, F)

window, stride = 16, 4
wins = x.unfold(dimension=0, size=window, step=stride)
print(wins.shape)

torch.Size([22, 9, 16])


6. 배치 행렬 연산: `matmul`, `bmm`, `einsum`
- for문으로 각 배치마다 matmul하던 작업을 한 줄로 수행할 수 있음

In [40]:
B, N, M, K = 32, 10, 20 ,8
A = torch.randn(B, N, K)
Bmat = torch.randn(B, K, M)

Y = torch.bmm(A, Bmat) # (B, N, M)
print(Y.shape)

torch.Size([32, 10, 20])


In [41]:
X = torch.randn(4, 5, 16)
Y = torch.randn(4, 7, 16)
dots = torch.einsum('bnd,bmd->bnm', X, Y)
print(dots.shape)

torch.Size([4, 5, 7])


- training 코드에서 실전 적용 예시

1. 배치별 loss를 for로 누적하여 한 번에 계산

In [ ]:
B, C = 100, 6
logits = torch.randn(B, C)
print(logits.shape)
labels = torch.tensor(torch.arange(B)) # (B,)
print(labels.shape)

torch.Size([1, 6])
torch.Size([1])


/tmp/ipykernel_161081/2055060249.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(torch.arange(B)) # (B,)


In [56]:
# bad
def bad():
    loss = 0.
    for i in range(B):
        loss += torch.nn.functional.cross_entropy(logits[i:i+1], labels[i:i+1])

%timeit bad

4.18 ns ± 0.0502 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [57]:
# good
loss = 0.
def good():
    loss = torch.nn.functional.cross_entropy(logits, labels)
%timeit good

4.14 ns ± 0.0357 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


2. 조건별 값 변경 : where/masked_fill

In [66]:
# bad: if/for
# good:
x = torch.randn(2, 3)
print(x)
x = x.masked_fill(x < 0, 0)
print(x)

tensor([[ 1.2381,  0.5523,  0.2781],
        [-0.8014,  0.0522, -0.3672]])
tensor([[1.2381, 0.5523, 0.2781],
        [0.0000, 0.0522, 0.0000]])
